In [1]:
import sys
sys.path.append("C:/code/qdc2")
import numpy as np, matplotlib.pyplot as plt
from qdc.mmf.many_wl_fiber import ManyWavelengthFiber
from qdc.mmf.qdc_mmf_experiment import QDCMMFExperiment

# Short illustration of phase matching induced mode mixing 

In [2]:
def make_exp(M, is_step_index=False):
    mwf = ManyWavelengthFiber(N_wl=3, npoints=2**8, is_step_index=is_step_index)
    exp = QDCMMFExperiment(mwf)
    exp.set_phase_matching(Lc_um=4000, pump_waist_crystal=500,
                           magnification=M, wl_pump=0.405, n_pump=1.692)
    return mwf, exp

def run_and_collect(M, is_step_index=False):
    mwf, exp = make_exp(M, is_step_index)
    mid = len(mwf.fibers) // 2
    f = mwf.fibers[mid]
    f.set_input_gaussian(*exp.g_params)

    E0 = f.propagate(show=False)
    E_filt = exp._apply_phase_matching(E0)

    n = exp.n
    G = exp._pm_pump_amp if exp._pm_pump_amp is not None else np.ones((n, n))
    S = np.fft.fftshift(exp._pm_filter)

    F0 = np.abs(np.fft.fftshift(np.fft.fft2(E0.reshape(n, n))))
    F1 = np.abs(np.fft.fftshift(np.fft.fft2(E_filt.reshape(n, n))))
    I0 = np.abs(E0.reshape(n, n)) ** 2
    I1 = np.abs(E_filt.reshape(n, n)) ** 2
    return dict(G=G, S=S, I0=I0, I1=I1, F0=F0, F1=F1, dx=mwf.dx)

# -------- plot helper with colorbar --------
def show(ax, img, ttl, dx=None, fiber_radius_um=None):
    extent = None
    if dx is not None:
        n = img.shape[0]
        extent = [-(n // 2) * dx, (n // 2) * dx,
                  -(n // 2) * dx, (n // 2) * dx]

    im = ax.imshow(img, extent=extent)
    ax.set_title(ttl, fontsize=8)
    # ax.axis('off')
    plt.colorbar(im, ax=ax, fraction=0.045, pad=0.02)

    # Add fiber core circle in real space
    if fiber_radius_um is not None and dx is not None:
        from matplotlib.patches import Circle
        circle = Circle((0, 0), radius=fiber_radius_um, edgecolor='red',
                        facecolor='none', lw=1)
        ax.add_patch(circle)

Ms = [3.3, 10]
is_step_index = True
data1 = run_and_collect(M=Ms[0], is_step_index=is_step_index)
data2 = run_and_collect(M=Ms[1], is_step_index=is_step_index)

titles = ['Gaussian', 'sinc PM', 'real |E|² before', 'real |E|² after',
          '|FFT| before', '|FFT| after']
rows = ['G', 'S', 'I0', 'I1', 'F0', 'F1']

fig, axes = plt.subplots(len(rows), 2, figsize=(9, 14))
for col, data, M in zip([0, 1], [data1, data2], Ms):
    dx = data['dx']
    for r, key in enumerate(rows):
        is_real_space = key in ['G', 'I0', 'I1']
        show(axes[r, col], data[key], f'{titles[r]} (M={M})',
             dx=dx if is_real_space else None,
             fiber_radius_um=25 if is_real_space else None)

plt.tight_layout()
plt.show()


Getting 3 fibers...


0it [00:00, ?it/s]C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\pyMMF\solvers\SI.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return jv(m, u) / (u * jv(m - 1, u)) + kn(m, w) / (w * kn(m - 1, w))
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\pyMMF\solvers\SI.py:147: RuntimeWarning: invalid value encountered in divide
  return jv(m, u) / (u * jv(m - 1, u)) + kn(m, w) / (w * kn(m - 1, w))
3it [00:31, 10.36s/it]


Got fibers!
Getting 3 fibers...


3it [00:01,  2.04it/s]


Got fibers!


# Illustration of different $\partial\beta_m / \partial\omega \cdot \Delta\omega\cdot L$ for different $m$ values 

In [ ]:
import sys
sys.path.append("C:/code/qdc2")
import numpy as np, matplotlib.pyplot as plt
from qdc.mmf.many_wl_fiber import ManyWavelengthFiber

is_step_index = True
mwf = ManyWavelengthFiber(wl0=0.810, Dwl=0.001, N_wl=3, npoints=2**8, is_step_index=is_step_index)
fig, ax = plt.subplots(2, 1, figsize=(10, 5))
ax[0].plot(range(len(mwf.betas[0])), mwf.betas[0], 'o-', label='lambda=0.810')
ax[0].plot(range(len(mwf.betas[1])), mwf.betas[1], 'o-', label='lambda=0.811')
ax[0].plot(range(len(mwf.betas[2])), mwf.betas[2], 'o-', label='lambda=0.812')
ax[0].legend()
ax[0].set_title('betas (1/um)')

diffs = np.diff(mwf.betas, axis=0)  # I get here the d_beta below and above, and see that they are similar 
c = 299792458
omegas = 2*np.pi*c/mwf.wls
Domega = omegas[1] - omegas[0]
# after 1m, d_beta/d_omega * Delta_omega * L, and I want to see how this differs between different modes, 
cutoff = 200 if is_step_index else 100
accumulated = diffs[:, :cutoff] * Domega * 0.1e6
ax[1].plot(range(len(accumulated[0])), accumulated[0], 'o-', label='diff1')
ax[1].plot(range(len(accumulated[1])), accumulated[1], 'o-', label='diff2')
ax[1].legend()
ax[1].set_title(r'$\partial\beta/\partial\omega \cdot \Delta\omega \cdot L$')
fig.show()

# MMF speckle envelope to normalize before PCC 

In [ ]:
import sys
sys.path.append("C:/code/qdc2")
import numpy as np, matplotlib.pyplot as plt
from qdc.mmf.many_wl_fiber import ManyWavelengthFiber
from qdc.mmf.fiber import Fiber

is_step_index = True

fiber_L = 0.2e6  if is_step_index else 3e6 # um 
N_wl = 81
N_classical = 5
N_SPDC = 5
wl0 = 0.810
Dwl = 0.010 if is_step_index else 0.020
NA_ref = 0.2   
dzs = [0, 10, 50, 200, 1000] if is_step_index else [0, 10, 50, 100]
npoints = 2**7
free_mode_matrix = False if npoints == 2**7 else True  # when working with 2**8, the RAM explodes 
autosolve = not free_mode_matrix  # if freeing each time - no point in autosolving initially 


n_pixels_diameter = npoints//5 if is_step_index else npoints//5
s = ManyWavelengthFiber(wl0=wl0, Dwl=Dwl, N_wl=N_wl, fiber_L=fiber_L, rng_seed=42, is_step_index=is_step_index, 
                        npoints=npoints, NA_ref=NA_ref, autosolve=autosolve)
s.gaussian_params = np.array([2.7, 7, 10, 0.5, 0.5]) 
s.gaussian_dparams = np.array([1, 5, 5, 0.3, 0.3])

f = s.fibers[0]

Getting 3 fibers...


0it [00:00, ?it/s]C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\pyMMF\solvers\SI.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return jv(m, u) / (u * jv(m - 1, u)) + kn(m, w) / (w * kn(m - 1, w))
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\pyMMF\solvers\SI.py:147: RuntimeWarning: invalid value encountered in divide
  return jv(m, u) / (u * jv(m - 1, u)) + kn(m, w) / (w * kn(m - 1, w))
3it [00:31, 10.66s/it]

Got fibers!


In [17]:
f.set_input_gaussian(*mwf.get_g_params())
f.propagate(show=False)
I_out = np.abs(f.profile_end)**2

for i in range(100):
    if i % 10 == 0:
        print(i)
    f.set_input_gaussian(*mwf.get_g_params())
    f.propagate(show=False)
    I_out += np.abs(f.profile_end)**2

fig, ax = plt.subplots(figsize=(10, 5))
imm = ax.imshow(I_out.reshape(f.npoints, f.npoints))
fig.colorbar(imm, ax=ax)
fig.show()

0
10
20
30
40
50
60
70
80
90
